## Checking SNP and haplotype genotyping error mismatch

I'm going a little crazy trying to understand how there could be such a discrepancy in genotyping error between the SNP and haplotype Genepop files, so I thought it was worth meticulously double-checking to make sure the error isn't me. It's usually me.

First, I'll take a look at the SNP Genepop file. It looks like this:

![image](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/snp_gp.png?raw=true)

In [1]:
cd /mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2

/mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2


I copied and pasted just the rows for the individuals with replicates, into this file 'cragigrun1_sampreps_snps_gen.txt', and it looks like this:

![image]()

In [ ]:
input = open("cragigrun1_sampreps_snps_gen.txt")
lines = input.readlines()
biglist = []
for line in lines:
    linelist = line.strip().split()
    biglist.append(linelist)
import numpy as np
myarray = np.array(biglist)

Assign rows of replicates to make small arrays w two rows each, for each replicated individual.

In [ ]:
FG101 = myarray[0:2]
FG102 = myarray[2:4]
FG103 = myarray[4:6]
FG104 = myarray[6:8]
tags_header = myarray[8]